In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("loan_severity (2).csv")

In [3]:
X_train = df.drop('target', axis = 1)
y_train = df['target']

In [4]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [5]:
df.head()

,DATA_RANGE_FROM,DATA_RANGE_TO,CONSENT_MODE,CONSENT_TYPE,FETCH_TYPE,FREQUENCY_VALUE,FI_TYPE,target,DIFFDAYS
0,2020-05-16,2027-12-16,store,transactions,one_time,9,deposit,0,2770 days
1,2023-12-17,2029-04-17,store,summary,periodic,6,deposit,0,1948 days
2,2018-10-21,2027-06-21,view,transactions,periodic,8,deposit,1,3165 days
3,2020-08-02,2023-05-02,query,transactions,one_time,5,deposit,0,1003 days
4,2023-11-06,2028-09-06,steam,transactions,periodic,8,deposit,0,1766 days


In [6]:
df['CONSENT_MODE'].unique()

array(['store', 'view', 'query', 'steam'], dtype=object)

In [7]:
class Drop_Label(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.label_encoder_consent_mode = LabelEncoder()
        self.label_encoder_consent_type = LabelEncoder()
        self.label_encoder_fetch_type = LabelEncoder()
        self.label_encoder_FI_type = LabelEncoder()
        
    def fit(self, X, y = None):
        self.label_encoder_consent_mode.fit(X['CONSENT_MODE'])
        self.label_encoder_consent_type.fit(X['CONSENT_TYPE'])
        self.label_encoder_fetch_type.fit(X['FETCH_TYPE'])
        self.label_encoder_FI_type.fit(X['FI_TYPE'])
        return self
        
    def transform(self, X, y = None):
        X = X.drop(['DATA_RANGE_FROM','DATA_RANGE_TO'], axis = 1)
        X['CONSENT_MODE'] = self.label_encoder_consent_mode.transform(X['CONSENT_MODE'])
        X['CONSENT_TYPE'] = self.label_encoder_consent_type.transform(X['CONSENT_TYPE'])
        X['FETCH_TYPE'] = self.label_encoder_fetch_type.transform(X['FETCH_TYPE'])
        X['FI_TYPE'] = self.label_encoder_FI_type.transform(X['FI_TYPE'])
        X['DIFFDAYS'] = X['DIFFDAYS'].map(lambda x: int(x.split(' ')[0]))
        return X
    

class one_hot(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.onehot_encoder_consent_mode = OneHotEncoder(handle_unknown='ignore',sparse = False)
        self.onehot_encoder_consent_type = OneHotEncoder(handle_unknown='ignore',sparse = False)
        self.onehot_encoder_fetch_type = OneHotEncoder(handle_unknown='ignore',sparse = False)
        self.onehot_encoder_FI_type = OneHotEncoder(handle_unknown='ignore',sparse = False)
        
    def fit(self, X, y = None):
        self.onehot_encoder_consent_mode.fit(X[['CONSENT_MODE']])
        self.onehot_encoder_consent_type.fit(X[['CONSENT_TYPE']])
        self.onehot_encoder_fetch_type.fit(X[['FETCH_TYPE']])
        self.onehot_encoder_FI_type.fit(X[['FI_TYPE']])
        return self
        
    def transform(self, X, y = None):
        enc_df = pd.DataFrame(self.onehot_encoder_consent_mode.transform(X[['CONSENT_MODE']]))
        col_names = []
        for x in enc_df.columns:
            col_names.append('CONSENT_MODE' +'_'+str(x))
        enc_df.columns = col_names
            
        X = X.join(enc_df)
        X = X.drop('CONSENT_MODE', axis = 1)
        
        enc_df = pd.DataFrame(self.onehot_encoder_consent_type.transform(X[['CONSENT_TYPE']]))
        col_names = []
        for x in enc_df.columns:
            col_names.append('CONSENT_TYPE' +'_'+str(x))
        enc_df.columns = col_names
            
        X = X.join(enc_df)
        X = X.drop('CONSENT_TYPE', axis = 1)
        
        enc_df = pd.DataFrame(self.onehot_encoder_fetch_type.transform(X[['FETCH_TYPE']]))
        col_names = []
        for x in enc_df.columns:
            col_names.append('FETCH_TYPE' +'_'+str(x))
        enc_df.columns = col_names
            
        X = X.join(enc_df)
        X = X.drop('FETCH_TYPE', axis = 1)
        
        enc_df = pd.DataFrame(self.onehot_encoder_FI_type.transform(X[['FI_TYPE']]))
        col_names = []
        for x in enc_df.columns:
            col_names.append('FI_TYPE' +'_'+str(x))
        enc_df.columns = col_names
            
        X = X.join(enc_df)
        X = X.drop('FI_TYPE', axis = 1)
        
        return X

In [8]:
pipe = Pipeline(steps = [
    ('1', Drop_Label()),
    ('2', one_hot()),
    ('scaler', MinMaxScaler())
])

In [9]:
X_train = pipe.fit_transform(X_train)
y_train = y_train.values

In [10]:
X_train.shape, y_train.shape

((100, 15), (100,))

### MODEL

In [17]:
import tensorflow as tf
import keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(6, activation='relu', input_shape = (15,)),
    Dense(5, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [18]:
model.compile(loss = 'binary_crossentropy', metrics = ['accuracy'], optimizer = 'adam')

In [19]:
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
4/4 [==============================] - 0s 26ms/step - loss: 0.7053 - accuracy: 0.3600
Epoch 2/100
4/4 [==============================] - 0s 17ms/step - loss: 0.7024 - accuracy: 0.3600
Epoch 3/100
4/4 [==============================] - 0s 14ms/step - loss: 0.7003 - accuracy: 0.4000
Epoch 4/100
4/4 [==============================] - 0s 8ms/step - loss: 0.6979 - accuracy: 0.4000
Epoch 5/100
4/4 [==============================] - 0s 8ms/step - loss: 0.6961 - accuracy: 0.4100
Epoch 6/100
4/4 [==============================] - 0s 6ms/step - loss: 0.6939 - accuracy: 0.4300
Epoch 7/100
4/4 [==============================] - 0s 8ms/step - loss: 0.6924 - accuracy: 0.4500
Epoch 8/100
4/4 [==============================] - 0s 8ms/step - loss: 0.6906 - accuracy: 0.4800
Epoch 9/100
4/4 [==============================] - 0s 10ms/step - loss: 0.6891 - accuracy: 0.5600
Epoch 10/100
4/4 [==============================] - 0s 10ms/step - loss: 0.6879 - accuracy: 0.5900
Epoch 11/100
4/4 [======

4/4 [==============================] - 0s 21ms/step - loss: 0.3621 - accuracy: 0.9200
Epoch 85/100
4/4 [==============================] - 0s 10ms/step - loss: 0.3547 - accuracy: 0.9300
Epoch 86/100
4/4 [==============================] - 0s 9ms/step - loss: 0.3471 - accuracy: 0.9400
Epoch 87/100
4/4 [==============================] - 0s 7ms/step - loss: 0.3402 - accuracy: 0.9300
Epoch 88/100
4/4 [==============================] - 0s 7ms/step - loss: 0.3333 - accuracy: 0.9300
Epoch 89/100
4/4 [==============================] - 0s 7ms/step - loss: 0.3264 - accuracy: 0.9500
Epoch 90/100
4/4 [==============================] - 0s 6ms/step - loss: 0.3203 - accuracy: 0.9500
Epoch 91/100
4/4 [==============================] - 0s 5ms/step - loss: 0.3142 - accuracy: 0.9500
Epoch 92/100
4/4 [==============================] - 0s 5ms/step - loss: 0.3079 - accuracy: 0.9400
Epoch 93/100
4/4 [==============================] - 0s 6ms/step - loss: 0.3013 - accuracy: 0.9400
Epoch 94/100
4/4 [=============

In [20]:
model.save('pao.h5')